In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.executable

'/opt/homebrew/anaconda3/envs/pret/bin/python3'

In [42]:
from metanno.stubs import TextComponent, TableComponent

In [51]:
from pret.stubs.joy import Box, Button, Tab, TabList, Tabs, Typography, Slider
from pret.stubs.react import div, use_state
from pret.render import component
from pret.state import proxy, use_tracked


def TextView():
    state = use_tracked(shared_state)
    selected, set_selected = use_state([])

    def on_mouse_select(modkeys, selection):
        if not len(selection):
            return
        set_selected(
            [
                *selected,
                *(
                    {
                        **span,
                    }
                    for span in selection
                ),
            ]
        )

    def on_key_press(key, modkeys, ranges):
        if key == "Backspace":
            shared_state['spans'] = [
                span for span in shared_state['spans']
                if not any(
                    ms['end'] >= span["begin"] and span['end'] >= ms['begin']
                    for ms in selected
                )
            ]
            set_selected([])
            return
        elif key == "a":
            label = "short"
        elif key == "b":
            label = "long"
        else:
            return

        for span in selected:
            shared_state['spans'].append({**span, "label": label, "id": f"{span['begin']}-{span['end']}-{label}"})

        set_selected([])

    return TextComponent(
        text="Le petit poucet est un livre pour enfant.\nLe Petit Poucet est un conte appartenant à la tradition orale, retranscrit et transformé par Charles Perrault en France et paru dans Les Contes de ma mère l'Oye, en 1697. C'est également le nom du personnage principal de ce conte.",
        spans=[{**span, "style": span["label"]} for span in state["spans"]],
        mouse_selection=selected,
        actions={},
        on_mouse_select=on_mouse_select,
        on_key_press=on_key_press,
        styles={
            "short": {"color": "pink"},
            "long": {"color": "black"},
        },
    )

@component
def TabPanel(*children, index=None, value=None):
    data = use_tracked(shared_state)

    def handle_click(e):
        shared_state["count"] += 1
        
    def handle_slide(e, new_value, old_value):
        shared_state["count"] = new_value

    return div(
        *(
            (
                Typography("Mon titre B", variant="h3"),
                Button(
                    f"mon bouton: {data['count']}",
                    #variant="contained",
                    color="primary",
                    disable_ripple=True,
                    on_click=handle_click,
                ),
                div(Slider(on_change=handle_slide, sx={"width": "100px", "margin": "10px"})),
                TextView(),
            )
            if (value == index)
            else ()
        ),
        # props
        role="tabpanel",
        hidden=index != value,
        id=f"simple-tabpanel-{index}",
        aria_labelledby=f"simple-tab-{index}",
    )

@component
def App():
    # count, set_count = use_state(0)
    index, set_index = use_state(0)

    def handle_index_change(e, new_index):
        set_index(new_index)

    return div(
        Box(
            Tabs(
                TabList(
                    Tab(
                        "Item One",
                        aria_controls="simple-tabpanel-0",
                        key="simple-tab-0",
                    ),
                    Tab(
                        "Item Two",
                        aria_controls="simple-tabpanel-1",
                        key="simple-tab-1",
                    ),
                    Tab(
                        "Item Three",
                        aria_controls="simple-tabpanel-2",
                        key="simple-tab-2",
                    ),
                ),
                default_value=index,
                on_change=handle_index_change,
                aria_label="basic tabs example",
            ),
            borderBottom=1,
            borderColor="divider",
        ),
        TabPanel(
            key="tab-panel-0",
            index=0,
            value=index,
        ),
        TabPanel(
            key="tab-panel-1",
            index=1,
            value=index,
        ),
    )


shared_state = proxy(
    {
        "count": 0,
        "count2": 0,
        "spans": [],
    },
    remote_sync=False,
)

#App()

In [52]:
App()

In [5]:
from pret.manager import get_manager

[autoreload of pret.manager failed: Traceback (most recent call last):
  File "/Users/perceval/Development/metanno/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/perceval/Development/metanno/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/perceval/Development/metanno/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/Users/perceval/Development/metanno/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 314, in update_class
    old_obj = getattr(old, key)
              ^^^^^^^^^^^^^^^^^
  File "/Users/perceval/Development/pret/pret/manager.py", line 62, in __get__
    self.instance = ref(instance)
                    ^^^^^^^^^^^^^
TypeError: cannot create weak reference to 'NoneType' object
]


In [2]:
from pret.mui import Button, Slider, Typography
from pret.react import div
#Slider(orientation='vertical')

In [3]:
from pret.state import proxy, get_manager
from pret.render import use_state, use_tracked, component, server_only
from pret.bridge import create_proxy
import addon
from pret.serialize import shared_pickler

In [4]:
state = proxy({
    "count": 0,
    "count2": 0,
}, remote_sync=True)

In [5]:
@component
def App():
    #count, set_count = use_state(0)
    data = use_tracked(state)
    
    def handle_click(e):
        state['count'] += 1
    
    return div(
        Typography("Mon titre", variant="h3"),
        Button(
            f"mon bouton: {data['count']}",
            variant="contained",
            color="primary",
            disable_ripple=True,
            on_click=handle_click,
        )
    );

@component
def App2():
    #count, set_count = use_state(0)
    data = use_tracked(state)
    
    def handle_click(e):
        state['count2'] += 1
    
    return div(
        Typography("Mon titre", variant="h3"),
        *(Button(
            f"mon bouton: {data['count2']}",
            variant="contained",
            color="primary",
            disable_ripple=True,
            on_click=handle_click,
        ) for i in range(data["count"]))
    );

In [5]:
#App()

In [6]:
#App2()

In [8]:
@component
def SliderApp():
    #count, set_count = use_state(0)
    data = use_tracked(state)
    
    def handle_change(event, value, active_thumb):
        print("ok")
        state['count'] = value
        pass
    
    return Slider(
        value=data['count'],
        variant="contained",
        color="primary",
        on_change=handle_change,
    );

In [2]:
#SliderApp()

In [9]:
@component
def SliderApp():
    #count, set_count = use_state(0)
    data = use_tracked(state)
    
    def handle_change(event, value, active_thumb):
        print("ok")
        state['count'] = value
        pass
    
    return Slider(
        value=data['count'],
        variant="contained",
        color="primary",
        on_change=handle_change,
    );

In [86]:
dico = {"a": "a"}

In [87]:
from weakref import finalize

In [11]:
state

{'count': 28}

In [10]:
from pret.render import rendered_view_refs

In [13]:
for ref in rendered_view_refs:
    try:
        print(len(ref().data))
    except:
        pass

57155


In [7]:
#App()

In [12]:
import gc
gc.collect()

DELETING VIEW <pret.render.PretView object at 0x10879e010>


0

In [ ]:
from pret.

In [18]:
del obj

12 19


NameError: name 'obj' is not defined

In [17]:
#App()

14 18


In [27]:
del pickler

In [28]:
import gc
gc.collect()

26 29
DELETED !


0

In [18]:
#App()

13 19


In [14]:
from pret.serialize import shared_pickler

In [15]:
from pret.react import p, b, h2, button, div

In [16]:
from pret.state import proxy, get_manager
from pret.render import use_state, use_tracked, component, server_only
import addon
from pret.serialize import shared_pickler

addon_dico = addon.__dict__

state = proxy({"foo": "mes valeurs", "nested": {"a": 1, "b": 2}}, remote_sync=True)

In [1]:
@server_only
def compute_value_on_server(x):
    return x + 1

@component
def MyApp(*children, server_func, **props):
    print(len(addon.__dict__))
    
    value, set_value = use_state(4)
    data = use_tracked(state, notify_in_sync=False)

    async def async_on_click(events):
        state["nested"]["a"] = await server_func(state["nested"]["a"])
        #state["nested"]["a"] += 1

    return div(
        h2("Mon titre"),
        b("En gras"),
        p(data["foo"]),
        button(f"cliquer pour augmenter: value vaut {data['nested']['a']}",
               on_click=async_on_click, ), )

#MyApp("ceci est mon texte", server_func=compute_value_on_server)

NameError: name 'server_only' is not defined

In [6]:
state["foo"] = "Voici un bouton !"

In [6]:
from pret.state import proxy, get_manager
from pret.render import use_state, use_tracked, component, server_only, P, H1, Button
import addon

addon_dico = addon.__dict__

state = proxy({"foo": "mes valeurs", "nested": {"a": 1, "b": 2}}, remote_sync=True)

@server_only
def compute_value_on_server(x):
    return x + 1

@component
def MyApp(*children, server_func, **props):
    print(len(addon.__dict__))
    
    value, set_value = use_state(4)
    data = use_tracked(state, notify_in_sync=False)

    async def async_on_click(events):
        state["nested"]["a"] = await server_func(state["nested"]["a"])
        #state["nested"]["a"] += 1

    return P(
        H1("Mon titre"),
        data["foo"],
        Button(f"cliquer pour augmenter: value vaut {data['nested']['a']}",
               on_click=async_on_click, ), )

#MyApp("ceci est mon texte", server_func=compute_value_on_server).reload('dump.pkl')

ImportError: cannot import name 'P' from 'pret.render' (/Users/perceval/Development/pret/pret/render.py)

In [2]:
import base64
from pret.serialize import dumps

class Dumper:
    def __init__(self, x):
        self.x = x
        
    def _repr_mimebundle_(self, *args, **kwargs):
        plaintext = repr(self)
        serialized = base64.encodebytes(
            (
                dumps(
                    self.x,
                    pickled_modules=(
                        "pret",
                        "pret.serialize",
                        "pret.state",
                        "pret.render",
                        "pret.bridge",
                        "pret.manager",
                        "addon",
                    ),
                    recurse=True,
                )
            )
        ).decode()
        print("SERIALIZED", len(serialized))
        
        if len(plaintext) > 110:
            plaintext = plaintext[:110] + "…"
        return {
            "text/plain": plaintext,
            "application/vnd.pret+json": {
                "detach": False,
                "version_major": 0,
                "version_minor": 0,
                "view_data": {
                    "serialized": serialized,
                    "bundled_modules": {},  # bundled_modules,
                },
            },
        }

In [3]:
import base64
import io
from pret.serialize import PretPickler
from dill import Unpickler as DillUnpickler, Pickler as DillPickler

dump_file = io.BytesIO()
shared_pickler = PretPickler(dump_file)

def dumps(
    obj,
    byref=None,
    recurse=None,
    no_recurse_in=None,
    pickled_modules=(),
):
    dump_file.seek(0)
    dump_file.truncate()

    shared_pickler.dump(obj)

    return dump_file.getvalue()

class Unpickler(DillUnpickler):
    """python's Unpickler extended to interpreter sessions and more types"""

    from dill.settings import settings

    def __init__(
        self, *, fix_imports=True, encoding="ASCII", errors="strict", buffers=None
    ):
        self.file = io.BytesIO()
        super().__init__(
            self.file,
            fix_imports=fix_imports,
            encoding=encoding,
            errors=errors,
            buffers=buffers,
        )

    def load_data(
        self, data, buffers=None
    ):  # NOTE: if settings change, need to update attributes
        # Add the content of file to self.file
        self.file.seek(0)
        self.file.truncate()
        self.file.write(data)
        self.file.seek(0)
        return self.load()


unpickler = Unpickler()

def loads(data):
    return unpickler.load_data(data)


import pickle
import io

class CustomClass:
    def __init__(self, value):
        self.value = value

# Create some objects
obj1 = CustomClass(1)
obj2 = CustomClass(2)
obj3 = obj1

x1 = dumps(obj1)
print(len(x1))

x2 = dumps(obj2)
print(len(x2))

x3 = dumps(obj1)
print(len(x3))

499
25
5


In [23]:
from pret.state import proxy, tracked, subscribe, snapshot, WeakValueDictionary, is_changed

class ProxiedClass:
    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls, *args, **kwargs)
        instance.__dict__['_state'] = proxy({"foo": "mes valeurs", "nested": {"a": 1, "b": 2}})
        return instance
    
    def subscribe(self, callback):
        return subscribe(self.__dict__['_state'], callback)
        
    def __getattr__(self, attr):
        try:
            return self.__dict__['_state'][attr]
        except KeyError:
            raise AttributeError(attr)
    
    def __setattr__(self, attr, value):
        self.__dict__['_state'][attr] = value
        
    def __dir__(self):
        return tuple(self.__dict__['_state'])
        
class Zoo(ProxiedClass):
    def __init__(self):
        self.cows = 3
        self.dogs = 2
        
x = Zoo()
x.subscribe(print)

<function pret.state.subscribe.<locals>.unsubscribe()>

In [27]:
x.cows = 5

TRIGGER LISTENER ! ['__setitem__', ['cows'], 5]
[['__setitem__', ['cows'], 5]]


In [ ]:
from pret.state import patch

In [ ]:
state = proxy({"foo": "mes valeurs", "nested": {"a": 1, "b": 2}})
subscribe(state, print, notify_in_sync=True)
state

{'foo': 'mes valeurs', 'nested': {'a': 1, 'b': 2}}

In [ ]:
patch(state, [['__setitem__', ['foo'], {'ok': 1, 'ko': 0}]])

In [ ]:
state

In [ ]:
state

{'foo': {'ok': 1, 'ko': 0}, 'nested': {'a': 1, 'b': 2}}

In [ ]:
from pret.state import proxy, subscribe, tracked, snapshot, WeakKeyDictionary, is_changed

state = proxy({"foo": "mes valeurs", "nested": {"a": 1, "b": 2}})
subscribe(state, print, notify_in_sync=True)
affected = WeakKeyDictionary()
snap_1 = snapshot(state)
tracked_state_1 = tracked(snap_1, affected)
snap_2 = snapshot(state)
tracked_state_2 = tracked(snap_2, affected)
x = tracked_state_1["nested"]
print("AFFECTED !")
for a in affected.values():
    print(a)
print("DONE")
state["nested"]["a"] = 3
snap_2 = snapshot(state)

AFFECTED !
Affected({'nested'}, set())
DONE
[['__setitem__', ['nested', 'a'], 3]]
